In [1]:
%load_ext autoreload
%autoreload 2
import torch, rdkit
from pathlib import Path, sys
_THIS_FILE = Path(globals().get("__file__", Path.cwd())).resolve()
PROJECT_ROOT = _THIS_FILE.parent
sys.path.insert(0, str(PROJECT_ROOT))
from rdkit import Chem
from utils.utils import *
from utils.dataloader import *
from utils.loss_fn import *
from utils.eval import *

from tqdm import trange
device   = "cuda"

vocab = dataset.vocab
index_to_token = {idx: token for token, idx in vocab.items()}

{'[#C]': 0, '[#N]': 1, '[*H0]': 2, '[=*H0]': 3, '[=Branch]': 4, '[=C]': 5, '[=N]': 6, '[=O]': 7, '[=S]': 8, '[Branch]': 9, '[C]': 10, '[Cl]': 11, '[EOS]': 12, '[F]': 13, '[NH0+1]': 14, '[N]': 15, '[OH0-1]': 16, '[O]': 17, '[PAD]': 18, '[PH1]': 19, '[P]': 20, '[Ring1]': 21, '[Ring2]': 22, '[SOS]': 23, '[S]': 24, '[SiH0]': 25, '[pop]': 26}
27
tensor([ 2, 10, 17, 10,  2, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18])
tensor([23,  2, 10, 17, 10,  2, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18])
tensor([ 2, 10, 17, 10,  2, 12, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18])


In [2]:
def select_model(choice, latent):
    if choice == "Trans_MHA":
        from models.Trans_MHA import CVAE, PriorNet
        model    = CVAE(latent_dim=latent).cuda()
        model.decoder.cuda()
        prior = PriorNet(y_dim=3, latent_dim=latent).cuda()

    # Trans
    elif choice == "Trans":
        from models.Trans import CVAE, PriorNet
        model    = CVAE(latent_dim=latent).cuda()
        model.decoder.cuda()
        prior = PriorNet(y_dim=3, latent_dim=latent).cuda()

    # LSTM
    elif choice == "LSTM":
        from models.LSTM import CVAE, PriorNet
        model    = CVAE(latent_dim=latent).cuda()
        model.decoder.cuda()
        prior = PriorNet(y_dim=3, latent_dim=latent).cuda()

    # LSTM + MHA
    elif choice == "LSTM_MHA":
        from models.LSTM_MHA import CVAE, PriorNet
        model    = CVAE(latent_dim=latent).cuda()
        model.decoder.cuda()
        prior = PriorNet(y_dim=3, latent_dim=latent).cuda()

    return model, prior

def get_loss_fn(choice, latent):
    if choice == "Trans_MHA":
        loss_fn = ConditionalVAELoss(
            vocab_size=dataset.vocab_size,
            max_beta=0.1,
            anneal_steps=300,
            cyc_steps=200,
            num_cycles=6,
            free_bits=0.08,
            capacity_max=0.0,
            capacity_inc=0.0,
            gamma=0.8,
            prop_w=0.02,
            nce=0.1,
            sig_pen_p=0.003,
            sig_pen_q=0.003,
            imb=0.05,
            latent_dim=latent
        ).cuda()

    # Trans
    elif choice == "Trans":
        loss_fn = ConditionalVAELoss(
            vocab_size=dataset.vocab_size,
            max_beta=0.01,
            anneal_steps=300,
            cyc_steps=200,
            num_cycles=6,
            free_bits=0.06,
            capacity_max=0.0,
            capacity_inc=0.0,
            gamma=0.8,
            prop_w=0.02,
            nce=0.1,
            sig_pen_p=0,
            sig_pen_q=0,
            imb=0.05,
            latent_dim=latent
        ).cuda()

    # LSTM
    elif choice == "LSTM":
        loss_fn = ConditionalVAELoss_LSTM(
            vocab_size=dataset.vocab_size,
            max_beta=0.1,
            anneal_steps=300,
            cyc_steps=200,
            num_cycles=6,
            free_bits=0.08,
            capacity_max=0.0,
            capacity_inc=0.0,
            gamma=0.8,
            prop_w=0.02,
            nce=0.1,
            sig_pen_p=0.003,
            sig_pen_q=0.003,
            imb=0.05,
            latent_dim=latent
        ).cuda()

    # LSTM + MHA
    elif choice == "LSTM_MHA":
        loss_fn = ConditionalVAELoss(
            vocab_size=dataset.vocab_size,
            max_beta=0.4,
            anneal_steps=300,
            cyc_steps=200,
            num_cycles=6,
            free_bits=0.08,
            capacity_max=0.0,
            capacity_inc=0.0,
            gamma=0.8,
            prop_w=0.02,
            nce=0.1,
            sig_pen_p=0.003,
            sig_pen_q=0.003,
            imb=0.05,
            latent_dim=latent
        ).cuda()
        
    return loss_fn

In [3]:
mode = "Trans"
latent_dim = 128
seq_len = dataset.max_len+3
model, prior = select_model(mode, latent_dim)
loss_fn = get_loss_fn(mode, latent_dim)

for head in [model.to_prop, model.to_prop_z]:
    nn.init.xavier_uniform_(head.weight)
    nn.init.zeros_(head.bias)

prop_params = list(model.to_prop.parameters())+list(model.to_prop_z.parameters())
prop_params_id = {id(p) for p in prop_params}
base_params = [p for p in model.parameters() if id(p) not in prop_params_id]
lr = 3e-4
lr_prior = lr*0.1
from torch.optim import AdamW
optim = AdamW([{'params':base_params, 'lr':lr},{'params':prop_params, 'lr':lr*0.5}])
optim2 = AdamW(prior.parameters(), lr=lr_prior)

In [ ]:
import datetime
from rdkit import Chem, RDLogger
RDLogger.DisableLog('rdApp.error')
status_out = widgets.Output()

display(status_out)
epoch = 1200
model.train()
prior.train()
progress = tqdm(range(epoch), desc="Training")
loss_arr=[]

for i in progress:
    log_var_extract=[]
    log_var_p_extract = []
    batchloss = 0.0
    embeddings = []
    mean_extract = []
    results = []
    kld_raw_batch = 0.0
    bce = 0.0
    kld = 0.0
    if mode=="Trans" or mode=="Trans_MHA":
      for (smiles_enc, smiles_dec_input, smiles_dec_output, properties) in train_dataloader:
        optim.zero_grad()
        optim2.zero_grad()

        smiles_enc = smiles_enc.to(device)
        smiles_dec_input = smiles_dec_input.to(device)
        smiles_dec_output = smiles_dec_output.to(device)
        properties = properties.to(device)
        output, tgt, means, log_var, tgt_z = model.forward(smiles_enc, smiles_dec_input, properties)
    
        mu_p, logvar_p = prior.forward(properties.squeeze())
        with torch.no_grad():
          log_var_extract.append(torch.exp(0.5 * log_var.cpu()).to(device))
          log_var_p_extract.append(torch.exp(0.5 * logvar_p.cpu()).to(device))
          mean_extract.append(means.cpu())
        
        loss, BCE, KLD, kld_raw, kld_per_token, prop = loss_fn.forward(output.float(),
                                smiles_dec_output,
                                means, log_var, mu_p, logvar_p,
                                tgt, properties.float().squeeze(), tgt_z,
                                i)
        
        results.append(output)

        loss.backward()
        optim.step()
        optim2.step()

        batchloss += loss.item()
        kld_raw_batch += kld_raw.item()
        bce += BCE.item()
        kld += KLD.item()
    else:
      for (smiles_dec_input, smiles_dec_output, properties) in train_dataloader_LSTM:
        tf_ratio = max(0.1, 1.0 - i / 300)
        optim.zero_grad()
        optim2.zero_grad()

        smiles_dec_input = smiles_dec_input.to(device)
        smiles_dec_output = smiles_dec_output.to(device)
        properties = properties.to(device)

        output, tgt, means, log_var, tgt_z = model.forward(smiles_dec_input, smiles_dec_output, properties, tf_ratio)
        
        mu_p, logvar_p = prior.forward(properties.squeeze())
        with torch.no_grad():
          log_var_extract.append(torch.exp(0.5 * log_var.cpu()).to(device))
          log_var_p_extract.append(torch.exp(0.5 * logvar_p.cpu()).to(device))
          mean_extract.append(means.cpu())
      
        loss, BCE, KLD, kld_raw, kld_per_token, prop = loss_fn.forward(output.float(),
                              smiles_dec_output,
                              means, log_var, mu_p, logvar_p,
                              tgt, properties.float().squeeze(), tgt_z,
                              i)
        loss.backward()
        optim.step()
        optim2.step()

        batchloss += loss.item()
        kld_raw_batch += kld_raw.item()
        bce += BCE.item()
        kld += KLD.item()
        
    #Loss 값 추가
    loss = batchloss / len(train_dataloader)
    loss_arr.append(loss)

    # #Validity
    # results = torch.cat(results, dim=0).cpu()
    # results = nn.functional.softmax(results, dim=-1) 
    # argmax_indices = torch.argmax(results, dim=-1)
    
    # valid_smiles = []
    # for row in argmax_indices:
    #    smiles = tok_ids_to_smiles(row.tolist())
    #    valid_smiles.append(smiles or "")

    # valid_count = sum(bool(s) for s in valid_smiles)
    # valid_frac  = valid_count / len(valid_smiles)

    # 진행 바의 속성으로부터 필요한 값들 추출 (예시)
    elapsed = int(progress.format_dict.get("elapsed", 0))
    formatted_elap = str(datetime.timedelta(seconds=elapsed))
    rate = progress.format_dict.get("rate", None)
    sec_per_iter = 1 / rate if rate and rate != 0 else 0
    total = int(sec_per_iter * progress.total)
    formatted_total = str(datetime.timedelta(seconds=total))
    
    # 고정된 상태 정보를 업데이트 (Output 위젯에 출력)
    with status_out:
        clear_output(wait=True)
        print(f"🔹 Elapsed: {formatted_elap} | sec/iter: {sec_per_iter:.3f}s")
        print("🔹 Total time: ", formatted_total)
        print(f"🔹 Step: {i+1}/{progress.total}")
        print("🔹 loss: {:0.6f}".format(loss))
        print("[Posterior] sigma mean : {:0.6f}, ".format(torch.cat(log_var_extract).mean().item()),
              "sigma std : {:0.6f}".format(torch.cat(log_var_extract).std().mean()))
        print("[Prior]     sigma mean : {:0.6f}, ".format(torch.cat(log_var_p_extract).mean().item()),
              "sigma std : {:0.6f}".format(torch.cat(log_var_p_extract).std().mean()))
        print("raw KLD : ", kld_raw_batch / len(train_dataloader))
        print("Posterior μ per-dim std:", torch.cat(mean_extract,dim=0).view(-1, latent_dim).std(dim=0, unbiased=False).detach().cpu().numpy().mean())
        print("Posterior μ overall std:", torch.cat(mean_extract,dim=0).view(-1, latent_dim).std(unbiased=False).item())
        print("BCE : {:0.6f},".format(bce / seq_len / len(train_dataloader)),
              "KLD : {:0.6f},".format(kld / seq_len / len(train_dataloader)),
              "prop : {:0.6f},".format(prop))
        # print(f"Validity: {valid_frac:.2%}  ({valid_count}/{len(valid_smiles)})")

Output()

Training:   0%|          | 0/1200 [00:00<?, ?it/s]

In [ ]:
model.eval().cuda()
prior.eval().cuda()
results = []
origin = []

properties_results=[]
properties_origin=[]
print(len(val_dataset))
with torch.no_grad():
    try:
        for (smiles_enc, smiles_dec_input, smiles_dec_output, properties) in val_dataloader:
            B = smiles_enc.size(0)

            smiles_enc = smiles_enc.to(device)
            smiles_dec_input = smiles_dec_input.to(device)
            smiles_dec_output = smiles_dec_output.to(device)
            properties = properties.to(device)

            result, tgt, means, log_var, z = model(smiles_enc, smiles_dec_input, properties)
            # result = model.deocder(smiles_dec_input, means.view(B, -1, latent_dim))
            # result = model.predict(result)

            results.append(result)
            origin.append(smiles_dec_output)

            properties_results.append(tgt)
            properties_origin.append(properties)
    except:
        for (smiles_dec_input, smiles_dec_output, properties) in val_dataloader_LSTM:

            smiles_dec_input = smiles_dec_input.to(device)
            smiles_dec_output = smiles_dec_output.to(device)
            properties = properties.to(device)

            result, tgt, means, log_var, z = model(smiles_dec_input, smiles_dec_output, properties)

            results.append(result)
            origin.append(smiles_dec_output)

            properties_results.append(tgt)
            properties_origin.append(properties)

results = torch.cat(results, dim=0)
origin = torch.cat(origin, dim=0)
results = nn.functional.softmax(results, dim=-1) 
argmax_indices = torch.argmax(results, dim=-1)
output = torch.nn.functional.one_hot(argmax_indices, num_classes=results.size(-1))
print(argmax_indices)
print(results.shape)
print(origin.shape)

from sklearn.metrics import mean_absolute_error
properties_origin=torch.cat(properties_origin,dim=0)
properties_results=torch.cat(properties_results,dim=0)
MAE_2 = mean_absolute_error(properties_origin.squeeze().cpu(), properties_results.squeeze().cpu())
print("MAE(properties) : ", MAE_2)


results_smiles = []
origin_smiles = []

for row in argmax_indices:
    smiles = tok_ids_to_smiles(row.tolist(), index_to_token)
    results_smiles.append(smiles or "")

for row in origin:
    smiles = tok_ids_to_smiles(row.tolist(), index_to_token)
    origin_smiles.append(smiles or "")

1567
tensor([[ 2, 10,  9,  ..., 12, 12, 10],
        [ 2, 15, 10,  ..., 10, 10, 10],
        [ 2, 17, 10,  ..., 12, 12, 12],
        ...,
        [ 2, 15, 10,  ..., 12, 12, 12],
        [ 2, 10, 15,  ..., 12, 12, 12],
        [ 2, 10, 15,  ..., 10, 10, 10]], device='cuda:0')
torch.Size([1567, 40, 27])
torch.Size([1567, 40])
MAE(properties) :  0.1713250263273237


In [ ]:
origin_smiles = [smiles.removesuffix("EOS").strip() for smiles in origin_smiles]
results_smiles = [smiles.removesuffix("EOS").strip() for smiles in results_smiles]

for i in range(len(results_smiles)):
    if(origin_smiles[i] != results_smiles[i]):
        print(i, "번째 다름!")
    print("real smiles      : ", origin_smiles[i])
    print("predicted smiles : ", results_smiles[i])


MAE = mean_absolute_error(origin.cpu(), torch.argmax(results.cpu(), dim=-1))
print("MAE : ", MAE)



0 번째 다름!
real smiles      :  [*]CC(C)OC(=O)OCC(C)N([*])CCC
predicted smiles :  [*]C(C)OC(=O)OC(C)N([*])CCC
1 번째 다름!
real smiles      :  [*]NC(=O)OC(CC)C([*])C(=O)OCC
predicted smiles :  [*]NC(=O)OC(C)C([*])C(=O)OC
2 번째 다름!
real smiles      :  [*]OC(=O)OCC(C)NC(=O)C(C)C([*])CC
predicted smiles :  *OC(=O)OC(C)NC(=O)C(*)C(*)C
3 번째 다름!
real smiles      :  [*]CNC(=O)CCCCNC(=O)NC([*])CC
predicted smiles :  [*]CNC(=O)CCCNNC(=O)NC([*])C
4 번째 다름!
real smiles      :  [*]COC(=O)NC([*])C(=O)NCCCC
predicted smiles :  [*]COC(=O)NC([*])C(=O)NCCC
5 번째 다름!
real smiles      :  [*]CNC(=O)COC(=O)OCC([*])C(C)C
predicted smiles :  [*]CNC(=O)COC(=O)OC([*])C(C)C
6 번째 다름!
real smiles      :  [*]OC(=O)OCC(O)=C(O)C([*])C=O
predicted smiles :  [*]OC(=O)C(O)=C(O)OC([*])=O
7 번째 다름!
real smiles      :  [*]COC(=O)OCCCCC(=O)NC([*])C=C
predicted smiles :  [*]COC(=O)OCCCC(O)NC([*])C=C
8 번째 다름!
real smiles      :  [*]CN(C)CCCNC(=O)OC([*])CSC
predicted smiles :  [*]CN(C)CCNNC(=O)OC([*])CSC
real smiles      :  [*]CNC(=O)OC

In [ ]:
from rdkit import Chem, RDLogger
from rdkit.Chem import DataStructs, rdFingerprintGenerator
RDLogger.DisableLog('rdApp.error')

generator = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=2048)

def tanimoto_similarity(smiles1, smiles2):
    mol1 = Chem.MolFromSmiles(smiles1)
    mol2 = Chem.MolFromSmiles(smiles2)
    fp1 = generator.GetFingerprint(mol1)
    fp2 = generator.GetFingerprint(mol2)
    return DataStructs.TanimotoSimilarity(fp1, fp2)

def is_valid(smiles):
    return Chem.MolFromSmiles(smiles) is not None

TS = 0.0
canbe = 0
notbe = 0

for sm, orig in zip(results_smiles, origin_smiles):
    if(len(sm)==0):
        notbe += 1
        continue
    try:
        sm = PS(sm).canonicalize.psmiles
        if is_valid(sm) and is_valid(orig):
            sim = tanimoto_similarity(sm, orig)
            TS += sim
            canbe += 1
        else:
            notbe += 1
    except:
        notbe += 1
        continue


if canbe > 0:
    print("Tanimoto Similarity : ", TS / canbe)
else:
    print("No valid molecules to compare.")

print("가능한 분자 개수 :", canbe)
print("불가능한 분자 개수 :", notbe)
print("Valid fraction      :", canbe / len(results_smiles))


Tanimoto Similarity :  0.5429316563508505
가능한 분자 개수 : 1531
불가능한 분자 개수 : 36
Valid fraction      : 0.97702616464582


In [ ]:
def save_weights(choice):
    if choice == "Trans_MHA":
        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_dmodel256.pth")
        torch.save(model.state_dict(), save_path)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_prior.pth")        
        torch.save(prior.state_dict(), save_path)

    # Trans
    elif choice == "Trans":
        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_dmodel256_no_mha.pth")
        torch.save(model.state_dict(), save_path)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_prior_no_mha.pth")        
        torch.save(prior.state_dict(), save_path)
    # LSTM
    elif choice == "LSTM":
        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM.pth")        
        torch.save(model.state_dict(), save_path)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM_prior.pth")       
        torch.save(prior.state_dict(), save_path)


    # LSTM + MHA
    elif choice == "LSTM_MHA":
        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM_MHA.pth")        
        torch.save(model.state_dict(), save_path)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM_MHA_prior.pth")        
        torch.save(prior.state_dict(), save_path)

def select_model(choice, latent):
    if choice == "Trans_MHA":
        from models.Trans_MHA import CVAE, PriorNet
        model    = CVAE(latent_dim=latent).cuda().eval()
        model.decoder.cuda().eval
        prior = PriorNet(y_dim=3, latent_dim=latent).cuda().eval()

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_dmodel256.pth")
        state_dict = torch.load(save_path)
        model.load_state_dict(state_dict)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_prior.pth")
        state_dict = torch.load(save_path)
        prior.load_state_dict(state_dict)

    # Trans
    elif choice == "Trans":
        from models.Trans import CVAE, PriorNet
        model    = CVAE(latent_dim=latent).cuda().eval()
        model.decoder.cuda().eval()
        prior = PriorNet(y_dim=3, latent_dim=latent).cuda().eval()

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_dmodel256_no_mha.pth")
        state_dict = torch.load(save_path)
        model.load_state_dict(state_dict)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_prior_no_mha.pth")
        state_dict = torch.load(save_path)
        prior.load_state_dict(state_dict)
    # LSTM
    elif choice == "LSTM":
        from models.LSTM import CVAE, PriorNet
        model    = CVAE(latent_dim=latent).cuda().eval()
        model.decoder.cuda().eval()
        prior = PriorNet(y_dim=3, latent_dim=latent).cuda().eval()

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM.pth")
        state_dict = torch.load(save_path)
        model.load_state_dict(state_dict)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM_prior.pth")
        state_dict = torch.load(save_path)
        prior.load_state_dict(state_dict)


    # LSTM + MHA
    elif choice == "LSTM_MHA":
        from models.LSTM_MHA import CVAE, PriorNet
        model    = CVAE(latent_dim=latent).cuda().eval()
        model.decoder.cuda().eval()
        prior = PriorNet(y_dim=3, latent_dim=latent).cuda().eval()

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM_MHA.pth")
        state_dict = torch.load(save_path)
        model.load_state_dict(state_dict)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM_MHA_prior.pth")
        state_dict = torch.load(save_path)
        prior.load_state_dict(state_dict)
    

    return model, prior

In [ ]:
save_weights(mode)

In [ ]:
# 1) 길이 페널티 키워서 긴 후보 선호
print(reconstruct_zmu(model, val_dataloader, dataset.vocab,
                      beam_width=100, len_penalty=1.0, temperature=0.80,
                      max_len=dataset.max_len+3))

# 150/8 -> 0.4704

NameError: name 'reconstruct_zmu' is not defined